In [5]:
import json
import pandas as pd
from tqdm.notebook import tqdm
from functools import partial
from sqlalchemy import MetaData, Table, create_engine
from sqlalchemy.sql import text

from config import db_connection_string
from utils import tx_count_between_accounts, tx_count_between_grants
from iou_calculator import IoUCalculator


In [2]:
engine = create_engine(db_connection_string)

In [3]:
with open('raw_data/contributions.json', 'r') as file:
    contributions = json.load(file)

contributions = [x for x in contributions if x['grant']!='86' and x['success']]

In [4]:
def generate_suspicious_ratio_df(pairs, objects, iou_threshold_list, min_contribution_count_list):
    return pd.DataFrame(
        [
            [
                iou_threshold, min_contribution_count, 
                (
                    str(sum(1 if len(x['contributions']) >= min_contribution_count else 0 for x in objects.values())) + ',' +
                    str(len({pair[i] for pair in pairs for i in range(2) if pair[2] >= iou_threshold and pair[3] >= min_contribution_count})) + ',' +
                    str(len({pair[i] for pair in pairs for i in range(2) if pair[2] >= iou_threshold and pair[3] >= min_contribution_count and pair[4] >= 1}))
                )
            ]
            for iou_threshold in iou_threshold_list
            for min_contribution_count in min_contribution_count_list
        ],
        columns=['iou_threshold', 'min_contribution_count', 'suspicious_account_ratio']
    ).pivot(index='iou_threshold', columns='min_contribution_count', values='suspicious_account_ratio')


## account suspicious ratio(grant)

In [5]:
calculator = IoUCalculator(
    contributions, lambda x: x['subscription']['contributor_profile'], lambda x: x['grant'],
    lambda x, y: tx_count_between_accounts(x, y, engine)
)
pairs = calculator.compute_suspicious_pairs(0.5, 1)
generate_suspicious_ratio_df(pairs, calculator.objects, [round(x * 0.1, 1) for x in range(5, 11)], [1, 3, 5, 10, 15, 20])

min_contribution_count,1,3,5,10,15,20
iou_threshold,,,,,,
0.5,"20655,16754,2018","10937,6880,1356","8671,5087,1197","4824,2526,655","3145,1590,512","2307,1180,329"
0.6,"20655,14290,1536","10937,5119,1025","8671,4034,904","4824,1948,533","3145,1175,426","2307,853,260"
0.7,"20655,12604,1285","10937,4173,826","8671,3382,740","4824,1506,433","3145,878,363","2307,605,206"
0.8,"20655,11963,1085","10937,3532,626","8671,2928,562","4824,1268,355","3145,709,299","2307,472,170"
0.9,"20655,11300,926","10937,2869,467","8671,2334,411","4824,986,291","3145,538,241","2307,341,128"
1.0,"20655,10957,851","10937,2526,392","8671,1991,336","4824,770,228","3145,403,185","2307,233,88"


## account suspicious ratio(grant+date)

In [6]:
calculator = IoUCalculator(
    contributions, lambda x:x['subscription']['contributor_profile'], lambda x:(x['grant'], x['created_on'][:10]),
    lambda x, y: tx_count_between_accounts(x, y, engine)
)
pairs = calculator.compute_suspicious_pairs(0.5, 1)
generate_suspicious_ratio_df(pairs, calculator.objects, [round(x * 0.1, 1) for x in range(5, 11)], [1, 3, 5, 10, 15, 20])


min_contribution_count,1,3,5,10,15,20
iou_threshold,,,,,,
0.5,"20655,11932,1037","11210,4369,753","8866,3566,662","5002,1633,372","3351,922,273","2497,659,164"
0.6,"20655,9496,828","11210,3590,605","8866,3040,538","5002,1305,318","3351,715,238","2497,486,135"
0.7,"20655,8845,712","11210,3168,504","8866,2695,450","5002,1089,271","3351,582,213","2497,375,114"
0.8,"20655,8487,624","11210,2810,416","8866,2388,369","5002,952,234","3351,494,187","2497,307,95"
0.9,"20655,8028,527","11210,2351,319","8866,1971,281","5002,780,193","3351,392,149","2497,232,65"
1.0,"20655,7782,494","11210,2105,286","8866,1725,248","5002,645,170","3351,318,131","2497,174,55"


## grant suspicious ratio(account)

In [6]:
calculator = IoUCalculator(
    contributions, lambda x:x['grant'], lambda x:x['subscription']['contributor_profile'],
    lambda x, y: tx_count_between_grants(x, y, engine)
)
pairs = calculator.compute_suspicious_pairs(0.5, 1)
generate_suspicious_ratio_df(pairs, calculator.objects, [round(x * 0.1, 1) for x in range(5, 11)], [1, 3, 5, 10, 15, 20])


min_contribution_count,1,3,5,10,15,20
iou_threshold,,,,,,
0.5,"1433,551,105","1187,350,75","988,175,44","728,57,22","616,31,9","541,28,9"
0.6,"1433,384,48","1187,200,22","988,92,16","728,26,13","616,8,4","541,6,4"
0.7,"1433,278,15","1187,133,7","988,50,5","728,12,5","616,2,2","541,2,2"
0.8,"1433,230,10","1187,85,2","988,25,0","728,2,0","616,0,0","541,0,0"
0.9,"1433,203,8","1187,58,0","988,12,0","728,0,0","616,0,0","541,0,0"
1.0,"1433,203,8","1187,58,0","988,12,0","728,0,0","616,0,0","541,0,0"


## grant suspicious ratio(account+date)

In [7]:
calculator = IoUCalculator(
    contributions, lambda x:x['grant'], lambda x:(x['subscription']['contributor_profile'], x['created_on'][:10]),
    lambda x, y: tx_count_between_grants(x, y, engine)
)
pairs = calculator.compute_suspicious_pairs(0.5, 1)
generate_suspicious_ratio_df(pairs, calculator.objects, [round(x * 0.1, 1) for x in range(5, 11)], [1, 3, 5, 10, 15, 20])


min_contribution_count,1,3,5,10,15,20
iou_threshold,,,,,,
0.5,"1433,496,71","1244,375,65","1047,200,31","776,54,19","665,31,9","591,22,6"
0.6,"1433,343,42","1244,252,36","1047,95,7","776,16,5","665,5,0","591,0,0"
0.7,"1433,281,29","1244,195,23","1047,49,3","776,10,3","665,2,0","591,0,0"
0.8,"1433,233,10","1244,147,4","1047,35,0","776,4,0","665,0,0","591,0,0"
0.9,"1433,187,8","1244,101,2","1047,15,0","776,0,0","665,0,0","591,0,0"
1.0,"1433,187,8","1244,101,2","1047,15,0","776,0,0","665,0,0","591,0,0"
